In [2]:
import torch
import torchvision
import numpy as np
import sys

In [4]:
batch_size =256
train_iter = torchvision.datasets.FashionMNIST(root='./Datasets',
               train=True,download=True,transform=torchvision.transforms.ToTensor())
test_iter = torchvision.datasets.FashionMNIST(root='./Datasets',
               train=False,download=True,transform=torchvision.transforms.ToTensor())

In [14]:
num_inputs = 784
num_outputs = 10

w = torch.randn(num_inputs,num_outputs)
#w = torch.FloatTensor(np.random.normal(0,1,(num_inputs,num_outputs)))
b = torch.zeros(10)
print(w.dtype,b.dtype)
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

torch.float32 torch.float32


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [18]:
X = torch.tensor([[1,2,3],[4,5,6]])
print(X.size())
print(X.sum(0,keepdim=True).size())
print(X.sum(1,keepdim=True).size())

torch.Size([2, 3])
torch.Size([1, 3])
torch.Size([2, 1])


In [41]:
def softmax(X):
    X_exp = X.exp()  #X还要用，不能在原地计算  原地计算是exp_() 
    #=赋值在函数结束后不会改变变量值，exp_会彻底改变
    partition = X_exp.sum(dim=1,keepdim=True)
    print(X_exp.size(),partition.size())
    return X_exp/partition   #广播
X = torch.rand((2, 5))
X_prob = softmax(X)
print(X_prob, X_prob.sum(dim=1))
print(X_prob.sum())

torch.Size([2, 5]) torch.Size([2, 1])
tensor([[0.1976, 0.1246, 0.2787, 0.1709, 0.2282],
        [0.1974, 0.2076, 0.1845, 0.2241, 0.1864]]) tensor([1.0000, 1.0000])
tensor(2.)


In [ ]:
def net(X):
    return softmax(torch.mm(X.view(-1,num_inputs),w)+b)